In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder,StandardScaler
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [5]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
len(train)

550068

In [6]:
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000004,P00128942,M,46-50,7,B,2,1,1,11.0,NaN
1,1000009,P00113442,M,26-35,17,C,0,0,3,5.0,NaN
2,1000010,P00288442,F,36-45,1,B,4+,1,5,14.0,NaN
3,1000010,P00145342,F,36-45,1,B,4+,1,4,9.0,NaN
4,1000011,P00053842,F,26-35,1,C,1,0,4,5.0,12.0


In [7]:
train.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


In [8]:
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [9]:
train.isnull().sum() 


User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2            173638
Product_Category_3            383247
Purchase                           0
dtype: int64

In [10]:
test.isnull().sum()

User_ID                            0
Product_ID                         0
Gender                             0
Age                                0
Occupation                         0
City_Category                      0
Stay_In_Current_City_Years         0
Marital_Status                     0
Product_Category_1                 0
Product_Category_2             72344
Product_Category_3            162562
dtype: int64

In [11]:
len(train['Purchase'])


550068

In [12]:
b = ['Product_Category_2','Product_Category_3']
for i in b:
    exec("train.%s.fillna(train.%s.value_counts().idxmax(), inplace=True)" %(i,i))
    exec("test.%s.fillna(test.%s.value_counts().idxmax(), inplace=True)" %(i,i))

In [14]:
datanew= train.drop(['Purchase'],axis=1)
len(datanew)

550068

In [15]:
print(datanew.columns)
datanew.dtypes

Index(['User_ID', 'Product_ID', 'Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Product_Category_1',
       'Product_Category_2', 'Product_Category_3'],
      dtype='object')


User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
dtype: object

In [16]:
Y = train["Purchase"]
len(Y)

550068

In [17]:
encode=LabelEncoder()
datanew=datanew.apply(encode.fit_transform)
datanew.City_Category = encode.fit_transform(datanew.City_Category)
datanew.Gender = encode.fit_transform(datanew.Gender)
datanew.head()


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,0,672,0,0,10,0,2,0,2,6,12
1,0,2376,0,0,10,0,2,0,0,4,10
2,0,852,0,0,10,0,2,0,11,6,12
3,0,828,0,0,10,0,2,0,11,12,12
4,1,2734,1,6,16,2,4,0,7,6,12


In [18]:
encode=LabelEncoder()
test=test.apply(encode.fit_transform)
test.City_Category = encode.fit_transform(test.City_Category)
test.Gender = encode.fit_transform(test.Gender)
test.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,3,1144,1,4,7,1,2,1,0,9,12
1,8,994,1,2,17,2,0,0,2,3,12
2,9,2672,0,3,1,1,4,1,4,12,12
3,9,1299,0,3,1,1,4,1,3,7,12
4,10,519,0,2,1,2,1,0,3,3,8


In [19]:
datanew.Gender = pd.to_numeric(datanew.Gender)
datanew.Age = pd.to_numeric(datanew.Age)
datanew.Occupation = pd.to_numeric(datanew.Occupation)
datanew.City_Category = pd.to_numeric(datanew.City_Category)
datanew.Stay_In_Current_City_Years = pd.to_numeric(datanew.Stay_In_Current_City_Years)
datanew.Marital_Status = pd.to_numeric(datanew.Marital_Status)
datanew.Product_Category_1 = pd.to_numeric(datanew.Product_Category_1)
datanew.Product_Category_2 = pd.to_numeric(datanew.Product_Category_2)
datanew.Product_Category_3 = pd.to_numeric(datanew.Product_Category_3)

In [20]:
test.Gender = pd.to_numeric(test.Gender)
test.Age = pd.to_numeric(test.Age)
test.Occupation = pd.to_numeric(test.Occupation)
test.City_Category = pd.to_numeric(test.City_Category)
test.Stay_In_Current_City_Years = pd.to_numeric(test.Stay_In_Current_City_Years)
test.Marital_Status = pd.to_numeric(test.Marital_Status)
test.Product_Category_1 = pd.to_numeric(test.Product_Category_1)
test.Product_Category_2 = pd.to_numeric(test.Product_Category_2)
test.Product_Category_3 = pd.to_numeric(test.Product_Category_3)

In [21]:
SS = StandardScaler()
data1 = SS.fit_transform(datanew)


In [25]:
from sklearn.decomposition import PCA
pc = PCA(4)
principalComponents = pc.fit_transform(data1)
pc.explained_variance_ratio_
principalDf = pd.DataFrame(data = principalComponents, columns = ["component 1", "component 2", "component 3", "component 4"])

In [27]:
kf=KFold(20)
for a,b in kf.split(principalDf):
    X_train, X_test = data1[a],data1[b]
    y_train, y_test = Y[a],Y[b]

In [28]:
lr = LinearRegression()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
gbr = GradientBoostingRegressor()


In [29]:
fit1 = lr.fit(X_train,y_train)
fit2 = dtr.fit(X_train,y_train)
fit3 = rfr.fit(X_train,y_train)
fit4 = gbr.fit(X_train,y_train)

In [ ]:
print("Accuracy Score of Linear regression on train set",fit1.score(X_train,y_train)*100)
print("Accuracy Score of Decision Tree on train set",fit2.score(X_train,y_train)*100)
print("Accuracy Score of Random Forests on train set",fit3.score(X_train,y_train)*100)
print("Accuracy Score of Gradient Boosting on train set",fit4.score(X_train,y_train)*100)

Accuracy Score of Linear regression on train set 11.829233894211866
Accuracy Score of Decision Tree on train set 100.0


In [ ]:
print("Accuracy Score of Linear regression on test set",fit1.score(X_test,y_test)*100)
print("Accuracy Score of Decision Tree on test set",fit2.score(X_test,y_test)*100)
print("Accuracy Score of Random Forests on test set",fit3.score(X_test,y_test)*100)
print("Accuracy Score of Gradient Boosting on testset",fit4.score(X_test,y_test)*100)

In [225]:
y1=lr.predict(test)
y1

array([11312.24386424, 10832.30040483,  8395.72409608, ...,
       11646.21467738,  7613.8591472 ,  8975.19832071])

In [226]:
y2=dtr.predict(test)
y2

array([19467.,  8405.,  5361., ..., 19003., 23714.,  2751.])

In [227]:
y3=rfr.predict(test)
y3

array([12431.36, 10555.51,  4483.28, ..., 10970.14, 21216.69,  2347.57])

In [228]:
y4=gbr.predict(test)
y4

array([13779.86258235, 10912.63270212,  6238.28563604, ...,
       13045.93618064, 18734.26884991,  2711.66811972])